<a href="https://colab.research.google.com/github/thieuhy/AgenticAI_Business_SJSU/blob/Agentic-AI-in-Supply-Chain/Group_8_Agentic_AI_in_Supply_Chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd, numpy as np, os
from datetime import date, timedelta

os.makedirs("data", exist_ok=True)

# Create 90 days of fake sales data
days = [date(2025,1,1) + timedelta(days=i) for i in range(90)]
sales = pd.DataFrame({
    "date": days,
    "sku": "A-101",
    "qty_sold": np.random.randint(40, 60, len(days))
})
sales.to_csv("data/sales.csv", index=False)

# Create starting inventory
inventory = pd.DataFrame({
    "sku": ["A-101"],
    "opening_stock": [350]
})
inventory.to_csv("data/inventory.csv", index=False)

# Create parameter file
params = pd.DataFrame({
    "sku": ["A-101"],
    "unit_cost": [10],
    "holding_cost_per_day": [0.5],
    "stockout_cost": [5],
    "lead_time_days": [7],
    "min_order_qty": [100],
    "service_level": [0.95]
})
params.to_csv("data/params.csv", index=False)

print("✅ Created CSV files in /data folder:")
print("sales.csv, inventory.csv, params.csv")


✅ Created CSV files in /data folder:
sales.csv, inventory.csv, params.csv


In [ ]:
!ls data

inventory.csv  params.csv  sales.csv


In [ ]:
import pandas as pd

sales = pd.read_csv("data/sales.csv")
inventory = pd.read_csv("data/inventory.csv")
params = pd.read_csv("data/params.csv")

print("Sales sample:")
print(sales.head(), "\n")

print("Inventory:")
print(inventory, "\n")

print("Params:")
print(params)


Sales sample:
         date    sku  qty_sold
0  2025-01-01  A-101        53
1  2025-01-02  A-101        50
2  2025-01-03  A-101        50
3  2025-01-04  A-101        41
4  2025-01-05  A-101        40 

Inventory:
     sku  opening_stock
0  A-101            350 

Params:
     sku  unit_cost  holding_cost_per_day  stockout_cost  lead_time_days  \
0  A-101         10                   0.5              5               7   

   min_order_qty  service_level  
0            100           0.95  


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import random
from collections import deque
import os
from datetime import date, timedelta

# --- GLOBAL SIMULATION PARAMETERS ---
SKU_ID = "A-101"

# Inventory Policy Parameters (These are the variables students should adjust)
LEAD_TIME = 7                  # Days until an order arrives
REVIEW_PERIOD = 1              # Days between the agent making a decision
SERVICE_LEVEL = 0.95           # Target service level (e.g., 95%)
MIN_ORDER_QTY = 100            # Minimum quantity that must be ordered (MOQ)
ALPHA = 0.1                    # Alpha for Exponentially Weighted Moving Average (EWMA)

# Cost Parameters (Total cost is the objective function the agent minimizes)
HOLDING_COST_PER_UNIT = 0.50   # Cost to hold one unit for one day
STOCKOUT_COST_PER_UNIT = 5.00  # Penalty for one unit of unfulfilled demand
ORDER_COST_FIXED = 50.00       # Fixed cost per purchase order (PO) placed

# --- 1. DATA GENERATION & INITIALIZATION ---
os.makedirs("data", exist_ok=True)

sales_path = "data/sales.csv"
inv_path = "data/inventory.csv"
params_path = "data/params.csv"

if not (os.path.exists(sales_path) and os.path.exists(inv_path) and os.path.exists(params_path)):
    days = [date(2025,1,1) + timedelta(days=i) for i in range(90)]
    sales = pd.DataFrame({
        "date": days,
        "sku": "A-101",
        "qty_sold": np.random.randint(40, 60, len(days))
    })
    sales.to_csv(sales_path, index=False)

    inventory = pd.DataFrame({
        "sku": ["A-101"],
        "opening_stock": [350]
    })
    inventory.to_csv(inv_path, index=False)

    params = pd.DataFrame({
        "sku": ["A-101"],
        "unit_cost": [10],
        "holding_cost_per_day": [0.5],
        "stockout_cost": [5],
        "lead_time_days": [7],
        "min_order_qty": [100],
        "service_level": [0.95]
    })
    params.to_csv(params_path, index=False)

    print("Created CSV files in /data.")
else:
    print("CSV files already exist; skipping generation.")


class DemandDataGenerator:
    """Generates synthetic, slightly volatile daily demand data."""
    def __init__(self, days, base_demand=50, vol_factor=0.2):
        self.days = days
        self.base_demand = base_demand
        self.vol_factor = vol_factor

    def generate_demand(self):
        # Create a series with daily base demand and add random noise
        np.random.seed(42) # for reproducibility
        noise = np.random.normal(0, self.base_demand * self.vol_factor, self.days)
        demand = np.maximum(0, self.base_demand + noise).round().astype(int)

        df = pd.DataFrame({
            'Day': range(1, self.days + 1),
            'Demand': demand
        })
        return df

class InventoryAgent:
    """
    The core agent system for inventory replenishment.
    It forecasts demand, calculates safety stock, and makes daily order decisions.
    """
    def __init__(self, initial_inventory, sku_params, forecast_method="ewma"):
        self.sku_id = sku_params['sku_id']
        self.L = sku_params['lead_time']
        self.R = sku_params['review_period']
        self.SL = sku_params['service_level']
        self.MOQ = sku_params['min_order_qty']
        self.ALPHA = sku_params['alpha']
        self.method = forecast_method

        # Financials
        self.H_COST = sku_params['holding_cost']
        self.S_COST = sku_params['stockout_cost']
        self.O_COST = sku_params['order_cost']

        # Agent state
        self.inventory_on_hand = initial_inventory
        self.forecast_history = []
        self.error_history = []
        self.po_in_transit = deque() # Queue of (quantity, arrival_day)
        self.backorders = 0
        self.is_review_day = False

        # Determine the Z-score for the target service level (one-sided)
        self.Z = norm.ppf(self.SL)

    # --- 2. FORECASTING & SAFETY STOCK ---

    def forecast_demand(self, actual_demand):
        # Naive forecast if selected
        if self.method == "naive":
            forecast = actual_demand
        else:
            # EWMA method
            if not self.forecast_history:
                forecast = actual_demand
            else:
                last_demand = self.error_history[-1]['ActualDemand']
                last_forecast = self.forecast_history[-1]
                forecast = (self.ALPHA * last_demand) + ((1 - self.ALPHA) * last_forecast)

        self.forecast_history.append(forecast)
        return round(forecast)


    def calculate_safety_stock(self):
        """
        Computes safety stock based on demand forecast error and the Z-score.
        Uses the Standard Deviation of Forecast Error (RMSE) over the risk period.
        Risk period = Lead Time + Review Period (L+R).
        """

        # Need at least 5 days of error history to calculate reliable standard deviation
        if len(self.error_history) < 5:
            return 0

        # Calculate recent forecast errors (Absolute Error)
        errors = [abs(e['Error']) for e in self.error_history]

        # Calculate Standard Deviation of Error (StDev)
        stdev_error = np.std(errors)

        # Safety Stock Formula: SS = Z * StDev_Error * np.sqrt(self.L + self.R)
        safety_stock = self.Z * stdev_error * np.sqrt(self.L + self.R)

        return round(safety_stock)

    # --- 3. CORE DECISION LOGIC ---

    def make_decision(self, current_day, daily_demand, total_inventory):
        """
        The agent checks its projected inventory position and decides whether to order.
        """
        log = []

        # Check if it is a review day
        self.is_review_day = (current_day - 1) % self.R == 0
        if not self.is_review_day:
            log.append("REVIEW: NO - Not a review day.")
            return 0, log

        # --- Step 1: Forecast Demand for the Risk Period (L + R) ---
        # The agent needs to estimate how much inventory it will consume until the next order arrives.
        risk_period = self.L + self.R

        # Use the latest forecast (made yesterday) and project it forward
        daily_forecast = self.forecast_history[-1] if self.forecast_history else daily_demand # Use last forecast or current demand if first day
        forecast_risk_period = daily_forecast * risk_period

        log.append(f"FORECAST: Daily={daily_forecast}, Risk Period ({risk_period} days)={forecast_risk_period}")

        # --- Step 2: Calculate Target Inventory & Safety Stock ---
        safety_stock = self.calculate_safety_stock()
        # Target Inventory = Forecasted Usage during Risk Period + Safety Stock
        target_inventory = forecast_risk_period + safety_stock
        log.append(f"SAFETY STOCK: Z={self.Z:.2f}, SS={safety_stock}, Target Inventory={target_inventory}")

        # --- Step 3: Calculate Inventory Position (IP) ---
        # Inventory Position = IOH + PO_In_Transit - Backorders
        # Sum all quantities currently in transit
        total_in_transit = sum(q for q, arrival in self.po_in_transit)

        inventory_position = self.inventory_on_hand + total_in_transit - self.backorders
        log.append(f"INVENTORY POSITION: IOH={self.inventory_on_hand} + PO={total_in_transit} - BO={self.backorders} = {inventory_position}")

        # --- Step 4: Decision Rule (Order Up To Target) ---
        order_qty = target_inventory - inventory_position

        # Apply constraints: MOQ and minimum order quantity must be non-negative
        if order_qty > self.MOQ:
            # Order decision: Place an order
            order_qty = max(order_qty, self.MOQ) # Ensure it meets MOQ

            # Record the PO arrival date
            arrival_day = current_day + self.L
            self.po_in_transit.append((order_qty, arrival_day))
            log.append(f"DECISION: ORDER {order_qty}. IP ({inventory_position}) < Target ({target_inventory}). Arriving Day {arrival_day}")
            return order_qty, log
        else:
            # Wait decision: Do not order
            log.append(f"DECISION: WAIT (No Order). IP ({inventory_position}) >= Target ({target_inventory}) or Order Qty ({order_qty:.2f}) < MOQ ({self.MOQ}).")
            return 0, log

# --- 4. SIMULATION ENGINE ---

class SimulationEngine:
    """Manages the daily loop, inventory updates, cost tracking, and logging."""
    def __init__(self, agent, demand_data):
        self.agent = agent
        self.demand_data = demand_data
        self.metrics = {
            'holding_cost': 0,
            'stockout_cost': 0,
            'order_cost': 0,
            'total_demand': 0,
            'fulfilled_demand': 0,
            'stockout_count': 0,
            'daily_log': []
        }

    def run_day(self, day, daily_demand):
        log = [f"--- DAY {day}: Demand={daily_demand} ---"]

        # --- A. Check for incoming orders (Arrivals) ---
        newly_arrived_qty = 0
        if self.agent.po_in_transit and self.agent.po_in_transit[0][1] == day:
            qty, arrival = self.agent.po_in_transit.popleft()
            self.agent.inventory_on_hand += qty
            newly_arrived_qty = qty
            log.append(f"ARRIVAL: Received PO of {qty} units. IOH is now {self.agent.inventory_on_hand}.")

        # --- B. Process Demand & Stockouts/Backorders ---
        stockout_units_today = 0
        demand_to_fulfill = daily_demand
        self.metrics['total_demand'] += demand_to_fulfill

        # 1. Fulfill any existing backorders first
        if self.agent.backorders > 0:
            fulfill_bo = min(self.agent.backorders, self.agent.inventory_on_hand)
            self.agent.backorders -= fulfill_bo
            self.agent.inventory_on_hand -= fulfill_bo
            log.append(f"FULFILLMENT: Cleared {fulfill_bo} backorders. Remaining BO: {self.agent.backorders}.")

        # 2. Fulfill current day's demand
        if self.agent.inventory_on_hand >= demand_to_fulfill:
            # Full fulfillment
            self.agent.inventory_on_hand -= demand_to_fulfill
            self.metrics['fulfilled_demand'] += demand_to_fulfill
            log.append(f"FULFILLMENT: Fulfilled {demand_to_fulfill} units. IOH: {self.agent.inventory_on_hand}.")
        else:
            # Partial or Zero fulfillment (Stockout occurs)
            fulfilled = self.agent.inventory_on_hand
            stockout = demand_to_fulfill - fulfilled

            self.agent.inventory_on_hand = 0
            self.agent.backorders += stockout

            self.metrics['fulfilled_demand'] += fulfilled
            self.metrics['stockout_count'] += 1 # Track days with a stockout
            stockout_units_today = stockout

            log.append(f"STOCKOUT: Only {fulfilled} fulfilled. {stockout} units added to backorders (Total BO: {self.agent.backorders}).")

        # --- C. Agent Review and Decision ---
        # The agent makes its decision for the next order based on the current state.
        order_qty, agent_log = self.agent.make_decision(day, daily_demand, self.agent.inventory_on_hand)
        log.extend(agent_log)

        # --- D. Cost Tracking ---
        # Holding Cost: Based on End-of-Day Inventory
        holding_cost = self.agent.inventory_on_hand * self.agent.H_COST
        self.metrics['holding_cost'] += holding_cost

        # Stockout Cost: Based only on today's unfulfilled demand (new stockouts)
        stockout_cost_today = stockout_units_today * self.agent.S_COST
        self.metrics['stockout_cost'] += stockout_cost_today

        # Order Cost: Fixed cost when a PO is placed
        if order_qty > 0:
            order_cost = self.agent.O_COST
            self.metrics['order_cost'] += order_cost
        else:
            order_cost = 0

        log.append(f"COSTS: Holding=${holding_cost:.2f}, Stockout=${stockout_cost_today:.2f}, Order=${order_cost:.2f}. Total Orders In Transit: {len(self.agent.po_in_transit)}")

        # --- E. Forecast Error Update (Using the actual demand *of the previous day*) ---

        # If this isn't the first day, calculate error for yesterday's forecast
        if len(self.agent.forecast_history) > 0:
            # Find yesterday's demand
            yesterday_demand = self.demand_data.loc[self.demand_data['Day'] == day - 1, 'Demand'].iloc[0] if day > 1 else daily_demand

            # Use the forecast made two days ago to predict yesterday's demand
            # This is complex in a daily loop. Simpler: Use the *latest* forecast (made at t-1)
            latest_forecast = self.agent.forecast_history[-1]

            error = yesterday_demand - latest_forecast
            self.agent.error_history.append({
                'Day': day,
                'ActualDemand': yesterday_demand,
                'Forecast': latest_forecast,
                'Error': error
            })

        # --- F. Make new forecast for tomorrow's decision (using current actual demand) ---
        new_forecast = self.agent.forecast_demand(daily_demand)
        log.append(f"FORECAST UPDATE: New Forecast (for tomorrow's decision)={new_forecast}")


        self.metrics['daily_log'].append(log)

    def run_simulation(self):
        """Main loop to iterate through the simulation days."""
        print(f"--- Running Inventory Replenishment Simulation for {SIMULATION_DAYS} Days ---")
        print(f"SKU: {self.agent.sku_id}, Service Level: {self.agent.SL*100}%, Lead Time: {self.agent.L} days\n")

        for index, row in self.demand_data.iterrows():
            self.run_day(row['Day'], row['Demand'])

        self.print_results()

    def print_results(self):
        """Prints the final performance metrics."""

        total_cost = self.metrics['holding_cost'] + self.metrics['stockout_cost'] + self.metrics['order_cost']
        fill_rate = self.metrics['fulfilled_demand'] / self.metrics['total_demand'] if self.metrics['total_demand'] > 0 else 0

        print("\n" + "="*50)
        print("SIMULATION RESULTS")
        print("="*50)

        print(f"\n[ PERFORMANCE METRICS ]")
        print(f"Total Demand:              {self.metrics['total_demand']:,} units")
        print(f"Total Fulfilled:           {self.metrics['fulfilled_demand']:,} units")
        print(f"Fill Rate (Orders Met):    {fill_rate:.2%}")
        print(f"Total Stockout Days:       {self.metrics['stockout_count']}")
        print(f"Final Inventory On Hand:   {self.agent.inventory_on_hand} units")
        print(f"Final Backorders:   {self.agent.backorders} units")

        print(f"\n[ FINANCIAL METRICS ]")
        print(f"Total Holding Cost:        ${self.metrics['holding_cost']:.2f}")
        print(f"Total Stockout Cost:       ${self.metrics['stockout_cost']:.2f}")
        print(f"Total Order Cost (Fixed):  ${self.metrics['order_cost']:.2f}")
        print(f"TOTAL SYSTEM COST:         ${total_cost:.2f}")
        print("="*50)

        # Print Agent Log (only for a few key days)
        print("\n--- DAILY AGENT LOG (First 5 Days) ---")
        for log in self.metrics['daily_log'][:5]:
            for line in log:
                print(f"| {line}")
            print("-" * 25)

        print("\n--- DAILY AGENT LOG (Last 5 Days) ---")
        for log in self.metrics['daily_log'][-5:]:
            for line in log:
                print(f"| {line}")
            print("-" * 25)

        print("\n[ Analysis Task ]")
        print("Study how changing the SERVICE_LEVEL (e.g., from 0.90 to 0.99) and ALPHA affects the Total Cost.")


# --- 5. EXECUTION ---
if __name__ == "__main__":
    # Initialize Demand Data
    daily_demand_data = pd.read_csv("data/sales.csv")
    daily_demand_data = daily_demand_data.rename(columns={'qty_sold': 'Demand'})
    daily_demand_data['Day'] = range(1, len(daily_demand_data) + 1)
    SIMULATION_DAYS = len(daily_demand_data)

    # Define SKU parameters for the AgentRubric
    params = pd.read_csv("data/params.csv")
    p = params.loc[params['sku'] == "A-101"].iloc[0]

    sku_parameters = {
        'sku_id': "A-101",
        'lead_time': int(p.lead_time_days),
        'review_period': 1,
        'service_level': float(p.service_level),
        'min_order_qty': int(p.min_order_qty),
        'alpha': ALPHA,  # or p.alpha if you put it in your CSV
        'holding_cost': float(p.holding_cost_per_day),
        'stockout_cost': float(p.stockout_cost),
        'order_cost': ORDER_COST_FIXED
    }

    # Starting Inventory (e.g., a few weeks' worth of base demand)
    inv = pd.read_csv("data/inventory.csv")
    INITIAL_INVENTORY = int(inv.loc[inv['sku'] == "A-101", 'opening_stock'].iloc[0]) # Fixed FutureWarning

    # --- GUARDRAILS
    # 1. Service Level
    if not isinstance(sku_parameters['service_level'], float):
        raise ValueError("Guardrail: service_level must be a number")
    if sku_parameters['service_level'] <= 0.5 or sku_parameters['service_level'] >= 0.999:
        raise ValueError("Guardrail: service_level must be between 0.5 and 0.999")

    # 2. Lead Time
    if sku_parameters['lead_time'] < 0:
        raise ValueError("Guardrail: lead_time must be >= 0")

    # 3. Review Period
    if sku_parameters['review_period'] <= 0:
        raise ValueError("Guardrail: review_period must be >= 1")

    # 4. Minimum Order Quantity (MOQ)
    if sku_parameters['min_order_qty'] < 0:
        raise ValueError("Guardrail: min_order_qty must be >= 0")

    # 5. EWMA Alpha
    if not (0 < sku_parameters['alpha'] <= 1):
        raise ValueError("Guardrail: alpha must be between 0 and 1 (0 < α ≤ 1)")

    # 6. Cost Parameters
    if sku_parameters['holding_cost'] < 0:
        raise ValueError("Guardrail: holding_cost cannot be negative")
    if sku_parameters['stockout_cost'] < 0:
        raise ValueError("Guardrail: stockout_cost cannot be negative")
    if ORDER_COST_FIXED < 0:
        raise ValueError("Guardrail: order_cost cannot be negative")

    # 7. Initial Inventory
    if INITIAL_INVENTORY < 0:
        raise ValueError("Guardrail: initial inventory must be >= 0")

    # 8. Demand sanity check
    if (daily_demand_data["Demand"] < 0).any():
        raise ValueError("Guardrail: demand cannot contain negative values")

    # 9. MOQ too large warning (not an error)
    expected_daily_demand = daily_demand_data["Demand"].mean()
    risk_period = sku_parameters['lead_time'] + sku_parameters['review_period']
    expected_risk_usage = expected_daily_demand * risk_period

    if sku_parameters['min_order_qty'] > 3 * expected_risk_usage:
        print(f"Guardrail Warning: MOQ ({sku_parameters['min_order_qty']}) "
              f"is over 3× expected usage (~{int(expected_risk_usage)} units).")


    # Initialize Agent and Simulation
    agent = InventoryAgent(INITIAL_INVENTORY, sku_parameters)
    sim = SimulationEngine(agent, daily_demand_data)

    # Run the simulation
    sim.run_simulation()

    # --- Baseline Comparison ---
    print("\n=== BASELINE: Naive Forecast (yesterday = today) ===")
    agent_naive = InventoryAgent(INITIAL_INVENTORY, sku_parameters, forecast_method="naive")
    sim_naive = SimulationEngine(agent_naive, daily_demand_data)
    sim_naive.run_simulation()

    print("\n=== EWMA Forecast (alpha = 0.1) ===")
    agent_ewma = InventoryAgent(INITIAL_INVENTORY, sku_parameters, forecast_method="ewma")
    sim_ewma = SimulationEngine(agent_ewma, daily_demand_data)
    sim_ewma.run_simulation()

    # --- Comparison Summary ---
    total_cost_naive = (sim_naive.metrics['holding_cost'] +
                        sim_naive.metrics['stockout_cost'] +
                        sim_naive.metrics['order_cost'])
    total_cost_ewma = (sim_ewma.metrics['holding_cost'] +
                       sim_ewma.metrics['stockout_cost'] +
                       sim_ewma.metrics['order_cost'])

    fill_rate_naive = sim_naive.metrics['fulfilled_demand'] / sim_naive.metrics['total_demand']
    fill_rate_ewma = sim_ewma.metrics['fulfilled_demand'] / sim_ewma.metrics['total_demand']

    print("\n=== COMPARISON SUMMARY ===")
    print(f"Naive Forecast: Fill Rate = {fill_rate_naive:.2%}, Total Cost = ${total_cost_naive:.2f}")
    print(f"EWMA Forecast:  Fill Rate = {fill_rate_ewma:.2%}, Total Cost = ${total_cost_ewma:.2f}")

    if total_cost_ewma < total_cost_naive:
        print("Result: EWMA performs better by reducing total cost.")
    else:
        print("Result: Naive forecast performs similarly or better — EWMA may be too sensitive.")

CSV files already exist; skipping generation.
--- Running Inventory Replenishment Simulation for 90 Days ---
SKU: A-101, Service Level: 95.0%, Lead Time: 7 days


SIMULATION RESULTS

[ PERFORMANCE METRICS ]
Total Demand:              4,457 units
Total Fulfilled:           4,025.3511261580816 units
Fill Rate (Orders Met):    90.32%
Total Stockout Days:       16
Final Inventory On Hand:   10.592510622353757 units
Final Backorders:   0.0 units

[ FINANCIAL METRICS ]
Total Holding Cost:        $2087.00
Total Stockout Cost:       $2158.24
Total Order Cost (Fixed):  $1800.00
TOTAL SYSTEM COST:         $6045.25

--- DAILY AGENT LOG (First 5 Days) ---
| --- DAY 1: Demand=53 ---
| FULFILLMENT: Fulfilled 53 units. IOH: 297.
| FORECAST: Daily=53, Risk Period (8 days)=424
| SAFETY STOCK: Z=1.64, SS=0, Target Inventory=424
| INVENTORY POSITION: IOH=297 + PO=0 - BO=0 = 297
| DECISION: ORDER 127. IP (297) < Target (424). Arriving Day 8
| COSTS: Holding=$148.50, Stockout=$0.00, Order=$50.00. Total Ord